In [4]:
# Final Script

In [7]:
# import scrapy
import json
import pymongo
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [5]:
def email(string):
    if not string or len(string) < 2:  # Check if string is empty or too short
        return None  # Or return an appropriate fallback value
    
    try:
        r = int(string[:2], 16)  # Convert the first two characters to an integer
        email = ''.join([chr(int(string[i:i+2], 16) ^ r) for i in range(2, len(string), 2)])
        return email
    except ValueError as e:
        print(f"Error decoding email: {e}")
        return None

In [69]:
def get_details(link):
    response = requests.get(link)    
    txt = response.text
    soup_time = BeautifulSoup(txt,'html.parser')
    date = soup_time.find('p',{'class':'mb-no'}).text
    start = txt.find("Media Contact")+21
    end = txt[start:].find("SOURCE")
    required_text = txt[start:start+end]
    soup = BeautifulSoup(required_text, 'html.parser')
    get_details = []
    if len(soup.find_all('p'))>1:
        for para in soup.find_all('p'):
            final_text = str(para)
            start = final_text.find("email-protection#")
            end = final_text[start:].find('"')
            # print(final_text[start+17:start+end])
            email_final = email(final_text[start+17:start+end])
            if email_final:
                final_text = final_text.replace('\xa0', ' ').replace('[email protected]', email_final) + " "
                soup = BeautifulSoup(final_text,'html.parser')
                get_details.append(soup.text)
    else:
        final_text = required_text
        start = final_text.find("email-protection#")
        end = final_text[start:].find('"')
            # print(final_text[start+17:start+end])
        email_final = email(final_text[start+17:start+end])
        if email_final:
            final_text = final_text.replace('\xa0', ' ').replace('[email protected]', email_final) + " "
            soup = BeautifulSoup(final_text,'html.parser')
            get_details.append(soup.text)
        
    return {'date':date,'data':get_details,'text':soup_time.text}

In [70]:
df = pd.read_csv("DataLinks_Dt_13_04.News_links.csv")

In [74]:
df['link']=[f"https://www.prweb.com{x}" for x in df['link']]

In [76]:
# get_details(df['link'][8])